In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
import nltk

In [2]:
# downloading stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Loading data set
review_data=pd.read_csv('D:\\Harshit Sethi\\Data Science CreTEch Internship\\Sentiment analysis\\IMDB Dataset.csv')

In [4]:
# Basic Overview
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
review_data.shape

(50000, 2)

In [6]:
review_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
review_data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [8]:
review_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [9]:
review_data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [10]:
# handling the dummy variable

review_data['sentiment']=review_data['sentiment'].map({'positive':1,'negative':0}) 

In [11]:
review_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [12]:
import re
import nltk

from nltk.corpus import stopwords
# Function to clean a single review
def clean_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-letter characters (keep only a-z or A-Z) 
    
    # Convert to lowercase and split into words
    text = text.lower().split()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]
    
    # Join the cleaned words back into a single string
    return ' '.join(text)

# Apply the function to the entire dataset
review_data['clean_review'] = review_data['review'].apply(clean_text)

# Show cleaned version of first few reviews
review_data[['review', 'clean_review']].head()


,review,clean_review
0,One of the other reviewers has mentioned that ...,one reviewers mentioned watching oz episode ho...
1,A wonderful little production. <br /><br />The...,wonderful little production br br filming tech...
2,I thought this was a wonderful way to spend ti...,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,basically family little boy jake thinks zombie...
4,"Petter Mattei's ""Love in the Time of Money"" is...",petter mattei love time money visually stunnin...


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # use top 5000 most frequent words

# Apply to the cleaned review column
x= tfidf.fit_transform(review_data['clean_review']).toarray()

# Target variable
y= review_data['sentiment']

In [14]:
from sklearn.model_selection import train_test_split
#splitting the data 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [15]:
print(x_train.shape)

(40000, 5000)


In [16]:
print(x_test.shape)

(10000, 5000)


In [17]:
from sklearn.linear_model import LogisticRegression

#Creating and Training the model
model=LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [18]:
# checking training accuracy
train_accuracy=model.score(x_train,y_train)
print('Training Accuracy', train_accuracy)

Training Accuracy 0.913525


In [19]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Model Testing
y_pred=model.predict(x_test)
#accuracy
test_accuracy=accuracy_score(y_test,y_pred)
print('Test Accuracy', test_accuracy)

#confusion matrix
print('Confusion Matrix', confusion_matrix(y_test,y_pred))

# Classification Report
print('Classification Report', classification_report(y_test,y_pred))

Test Accuracy 0.8924
Confusion Matrix [[4358  603]
 [ 473 4566]]
Classification Report               precision    recall  f1-score   support

           0       0.90      0.88      0.89      4961
           1       0.88      0.91      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

